In [1]:
import numpy as np
import networkx as nx
import netgraph
import matplotlib.pylab as plt
import os
from tqdm import tqdm

os.makedirs("./Karate", exist_ok=True)

from Qommunity.samplers.hierarchical.advantage_sampler import AdvantageSampler
from Qommunity.samplers.regular.leiden_sampler import LeidenSampler
from Qommunity.samplers.regular.louvain_sampler import LouvainSampler
from Qommunity.samplers.regular.dqm_sampler import DQMSampler
#from Qommunity.samplers.regular.bayan_sampler import BayanSampler

from Qommunity.searchers.community_searcher.community_searcher import CommunitySearcher
from Qommunity.searchers.hierarchical_community_searcher import HierarchicalCommunitySearcher

from iterative_searcher.iterative_searcher import IterativeSearcher

In [3]:
K = nx.karate_club_graph()
num_runs = 50
resolution = 1

In [76]:
adv_sampler = AdvantageSampler(K, resolution=resolution, num_reads=100, use_clique_embedding=True)
adv_searcher = HierarchicalCommunitySearcher(adv_sampler)
adv_iterative= IterativeSearcher(adv_searcher)

saving_path = f"./Karate/solver-adv_runs-{num_runs}"
adv_communities, adv_modularities, adv_times_elapsed = adv_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|██████████| 50/50 [18:19<00:00, 21.99s/it]


In [35]:
louv_sampler = LouvainSampler(K, resolution=resolution)
louv_searcher = CommunitySearcher(louv_sampler)
louv_iterative= IterativeSearcher(louv_searcher)

saving_path = f"./Karate/solver-louv_runs-{num_runs}"
louv_communities, louv_modularities, louv_times_elapsed = louv_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|██████████| 50/50 [00:00<00:00, 221.11it/s]


In [36]:
dqm_sampler = DQMSampler(K, resolution=resolution, time=5)
dqm_searcher = CommunitySearcher(dqm_sampler)
dqm_iterative= IterativeSearcher(dqm_searcher)

saving_path = f"./Karate/solver-dqm_runs-{num_runs}"
dqm_communities, dqm_modularities, dqm_times_elapsed = dqm_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

  0%|          | 0/50 [00:00<?, ?it/s]/home/joanfr/Documents/Git-Repos/QHyper/QHyper/converter.py:193: ProblemWarning: Defined problem has constraints. DQM does not support constraints, it only supports objective functions!
  warnings.warn(
100%|██████████| 50/50 [14:32<00:00, 17.44s/it]


In [5]:
leid_sampler = LeidenSampler(K)
leid_searcher = CommunitySearcher(leid_sampler)
leid_iterative= IterativeSearcher(leid_searcher)

saving_path = f"./Karate/solver-leid_runs-{num_runs}"
leid_communities, leid_modularities, leid_times_elapsed = leid_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

100%|██████████| 50/50 [00:00<00:00, 363.78it/s]


In [ ]:
# TODO: Add Bayan, but I cannot install bayanpy
baya_sampler = BayanSampler(K)
baya_searcher = CommunitySearcher(baya_sampler)
baya_iterative= IterativeSearcher(baya_searcher)

saving_path = f"./Karate/solver-baya_runs-{num_runs}"
baya_communities, baya_modularities, baya_times_elapsed = baya_iterative.run(
    num_runs=num_runs, saving_path=saving_path
)

In [79]:
mod_adv, freq_adv = np.unique(adv_modularities, return_counts=True)
mod_dqm, freq_dqm = np.unique(dqm_modularities, return_counts=True)
mod_louv, freq_louv = np.unique(louv_modularities, return_counts=True)
mod_leid, freq_leid = np.unique(leid_modularities, return_counts=True)
#mod_baya, freq_baya = np.unique(baya_modularities, return_counts=True)

In [87]:
print("Algorithm \t Max Q \t\t Frequency \t Time (s)")
print("---------------------------------------------------------")
print(f"Annealing \t {round(mod_adv[-1],6)}  \t {100*freq_adv[-1]/num_runs} %  \t {adv_times_elapsed.mean()} +- {adv_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"DQM       \t {round(mod_dqm[-1],6)}  \t {100*freq_dqm[-1]/num_runs} %  \t {dqm_times_elapsed.mean()} +- {dqm_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"Louvain   \t {round(mod_louv[-1],6)} \t {100*freq_louv[-1]/num_runs} % \t {louv_times_elapsed.mean()} +- {louv_times_elapsed.std()/np.sqrt(num_runs)}")
print(f"Leiden    \t {round(mod_leid[-1],6)} \t {100*freq_leid[-1]/num_runs} % \t {leid_times_elapsed.mean()} +- {leid_times_elapsed.std()/np.sqrt(num_runs)}")
#print(f"Bayan    \t {round(mod_baya[-1],6)} \t {100*freq_baya[-1]/num_runs} % \t")

Algorithm 	 Max Q 		 Frequency 	 Time (s)
---------------------------------------------------------
Annealing 	 0.440181  	 100.0 %  	 21.987347683906556 +- 0.5197222820404348
DQM       	 0.399833  	 100.0 %  	 17.44094145298004 +- 1.2429913791803389
Louvain   	 0.444904 	 40.0 % 	 0.0037561511993408204 +- 0.0004049010300129798
Leiden    	 0.444904 	 100.0 % 	 0.002487516403198242 +- 7.50826605237824e-05


In [82]:
adv_times_elapsed

array([21.18876052, 19.37231827, 20.28167272, 18.55721092, 17.67141366,
       17.70617509, 17.55001092, 18.99513292, 19.00003576, 17.96673632,
       28.05179763, 18.89762545, 18.93175864, 19.34211612, 20.55273151,
       20.27295113, 19.8435905 , 18.53318882, 19.60020828, 19.62291741,
       18.8237536 , 19.29578257, 19.8704381 , 20.12062097, 31.70817327,
       19.9230299 , 21.83447051, 22.35311031, 21.59195733, 21.62448072,
       22.25279117, 23.24518991, 23.57013726, 23.0891118 , 23.92263794,
       23.39641905, 21.55712819, 23.45466208, 23.17518282, 22.58018661,
       37.56984973, 24.25155592, 23.40872908, 23.8043797 , 23.83733916,
       27.322016  , 24.34552073, 23.34079957, 26.88958001, 25.2699976 ])

In [8]:
mod_leid, freq_leid = np.unique(leid_modularities, return_counts=True)

In [15]:
leid_times_elapsed[leid_modularities.argmax()]

0.002042055130004883